# Geospatial Data Analysis: Vector data  

SnowEx Hackweek  
July 14, 2021

### Scratch notes
* Philosophy for this tutorial
    * Here's what will work, simple, classic examples
    * Try to avoid overwhelming, but give basic resources
    * Try to provide cookbook examples for plug and play 
        * Read code/comments/doc
    * Highlighting caveats:
        * Always more complex
        * Look at the code
    * Scaling issues
    * Command line, Python script or Notebook?
    * No "right" way
* Core properties:
    * Coordinate systems, datums
        * Common options, gotchas (if you see a 30 m vertical offset, don't panic)
        * Can use common, predefined coordinate systems (preferable) or define yur own
    * Projections
        * Micah's don't the hard work to prepare common projection for DB
        * Inevitably you will integrate other data here and in the future
        * Reprojection
            * to_crs
        * Key takeaway - plotting, sampling, datasets must have common projection
            * Raster affine/geotransform?
        * How to choose a projection?
            * https://projectionwizard.org/
* DataFrame analysis
    * Filtering
    * Calculating new fields
    * Spatial analysis
        * Buffer, intersect?
* Combining vector and raster data
    * Sampling raster or ndarray at points
        * https://github.com/uw-cryo/raster_sampling
    * Zonal statistics for polygons
    * Hypsometry and statistics for elevation bands - binning one raster/vector dataset by another, snow depth elevation, snow depth vs. slope
    * Clipping a raster by a polygon

## Introduction
This tutorial will attempt to cover key fundamentals of geospatial data analysis with Python, with emphasis on vector datasets.

This will be a high-level overview, with lots of material. We don't expect you to absorb all of this today, and a few hours is not nearly enough time to cover even the basics. We hope to show you enough to get you started, and we leave it to you to review on your own, or revisit as necessary during projects. There are no expectations for you to absorb or remember all of this, but hopefully you will know where to look the next time you encounter some of these topics. We've done entire hackweeks (GeoHackWeek) and graduate-level courses (GDA) on these topics, and even with 10 weeks, we are still only able to scratch the surface. 

We've also prepared some notebooks with more "cookbook" recipes for common geospatial analysis tasks, and other snippets that you may find useful for your projects. The hope is that you can review these on your own

# Objectives

* Review fundamental concepts in geospatial analysis (e.g. coordinate systems, projections, geometry types)
* Learn basic geospatial data manipulation, exploration, and visualization using common tools
* Explore different approaches for spatial filtering and aggregation

# Introduction

## Quick Zoom poll:
In the Zoom Participants panel, use the Yes (green) or No (red) response to answer the following question: 

**Have you ever taken a GIS course?**

## GIS Basics
>A geographic information system (GIS) is a framework for gathering, managing, and analyzing data. Rooted in the science of geography, GIS integrates many types of data. It analyzes spatial location and organizes layers of information into visualizations using maps and 3D scenes. ​With this unique capability, GIS reveals deeper insights into data, such as patterns, relationships, and situations—helping users make smarter decisions.

https://www.esri.com/en-us/what-is-gis/overview

### Primary geospatial data types
* Vector - points, lines, polygons
    * File types: shapefiles, csv, geojson, geopackage
    * https://automating-gis-processes.github.io/site/notebooks/L1/geometric-objects.html
* Raster - gridded data, images
    * File types: GeoTiff

### Common geospatial data concepts:
* Extent - bounding box or envelope of dataset (xmin, ymin, xmax, ymax)
* Coordinate Systems - map projection and datum: https://geopandas.org/projections.html
* Geometry manipulation/operations: 
    * https://geopandas.org/docs/user_guide/geometric_manipulations.html
    * https://geopandas.org/docs/user_guide/set_operations.html

## Common data types, examples, and packages
* static vector data
    * e.g., point data for snow pit locations and attributes
    * (geopandas, ogr)
* static raster data
    * e.g, DEM, multispectral image data from Landsat-8
    * rasterio, gdal
* vector time series
    * e.g., flight trajectory data (x,y,z,time)
    * (geopandas, xarray)
* raster time series 
    * Sentinel-1 SAR backscatter time series
    * (xarray, rioxarray, xarray-spatial)

## The Scientific Python landscape

* Python
* Jupyter/iPython
* NumPy, Pandas, Matplotlib, SciPy
* xarray, scikit-learn

One interpretation of this stack:

![2017 Scientific Python Stack](https://devopedia.org/images/article/60/7938.1587985662.jpg)  
Slide from Jake VanderPlas’s presentation at PyCon 2017, entitled “The Unexpected Effectiveness of Python in Science.”

## The Geospatial Python landscape
* GDAL/OGR, GEOS, PROJ
* rasterio, fiona, shapely, pyproj
* geopandas, cartopy, xarray
* rioxarray, xarray-spatial, PDAL

[Insert shiny new figure here]

## Many excellent resources available
Here are a few...

#### Full courses:
* https://www.earthdatascience.org/courses/
* https://automating-gis-processes.github.io
* https://github.com/UW-GDA/gda_course_2021

#### Pangeo:
* http://gallery.pangeo.io/

#### Geohackweek: 
* https://geohackweek.github.io/

## Complementary approaches for geospatial data

1. Higher-level data science - *rapid analysis, interpetation, and visualization*
    * Short, easy-to-understand code
    * Convenience and flexibility comes with small performance hit
    * Memory (RAM) limitations
    * Geopandas
1. Efficient, scalable processing of huge datasets
    * Parallel operations
    * Lazy evaluation
    * PostGIS database
    * Dask

Here, we're going to explore #1, which is sufficient for scientific analysis.

As with all things in the *nix/open-source/Python world, there are always multiple approaches that can be used to accomplish the same goals.  You, the user, will decide on the best approach based on complexity, time constraints, etc.

## Coordinate Reference Systems (CRS)
* https://docs.qgis.org/3.16/en/docs/gentle_gis_introduction/coordinate_reference_systems.html
> The topic map projection is very complex and even professionals who have studied geography, geodetics or any other GIS related science, often have problems with the correct definition of map projections and coordinate reference systems. Usually when you work with GIS, you already have projected data to start with. In most cases these data will be projected in a certain CRS, so you don’t have to create a new CRS or even re project the data from one CRS to another. That said, it is always useful to have an idea about what map projection and CRS means.
* https://datacarpentry.org/organization-geospatial/03-crs/
* https://www.earthdatascience.org/courses/earth-analytics/spatial-data-r/intro-to-coordinate-reference-systems/

## EPSG codes
* European Petroleum Survey Group
* https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset

### Notes
* Proj, why so complicated?
* Early survey, level, local geodetic - relative to gravitational equipotential surface (bubble level, water)
* Satellite era
* Geopandas (and most GIS operations) operate on 2D cartesian coordinate systems
    * Don't account for curvature of the Earth
    * Distortion increases with distance from projection center or standard parallels
* How to check crs of dataset
* GIS will dynamically reproject all layers to match the "project CRS"

### Projections
* Need a way to represent 3D surface of the Earth on a 2D surface (paper map, computer screen)
* Why don't we just do everything in lat/lon?
    * geodetic calculations are expensive, require iteration to solve nonlinear differntial equations
    * 2D cartesian calculations (e.g., Pythagorean theorem for distance) much simpler, faster

### Horizontal CRS and Vertical CRS
* Ellipsoid and geoid
* Simple shape model for the Earth
* Gravitational equipotential surface
    * Gravitational vector is normal to this surface everywhere, magnitude is identical
    * Define 0 at "sea level" if there were no continents

### Add figure comparing ECEF, geodetic and projected coordinate systems for Earth

### Common CRS
* EPSG:4326 - WGS84, geodetic latitude and longitude
* EPSG:3857 - web mercator, https://en.wikipedia.org/wiki/Web_Mercator_projection
* UTM zones (North)
    * EPSG:326XX - where XX is zone number (e.g., 12), WGS84 ellipsoid
    * EPSG:269XX - NAD83(2011) ellipsoid

### How to choose a projection
* https://projectionwizard.org/
* Global datasets
* Different types of distortion (distance, directions) for the different projections.
* It's important to pick a projection that is well-suited for your application.  If you care about accurate representation of distances, you should use an equidistant projection.  If you care about accurate representation of areas, use an equal-area projection.
* Remember, there's no "perfect" projection. Always some compromise
* You would be surprised at how often errors due to projection decisions end up in published literature.

### How to tell if you have a coordinate system and/or vertical datum problem
* Difference DEMs - Hmm, offset of 20-30 m
    * Vertical datum, likely height above ellipsoid vs. height above geoid
* Errors during sampling at points ("out of bounds")
    * Check to make sure projections are the same, mixing lat/lon (decimal degrees) and projected coords (UTM in meters), using pixel/array coords vs. map coords

### General Words of Wisdom
* Always sanity check - does your dataset have a CRS defined?
* Subsample or subset large datasets for your project, don't load 1 m lidar for everything
* Avoid unnecessary reprojection and resampling is possible - introducing uncertainty with each transformation, degradation of quality due to interpolation
* When plotting and doing anlaysis, all of your input datasets must share the same coordinate system, which may require reprojecting one (or more) datasets to match.  You can then plot them on the same axes.  We will do more of this in coming weeks to create plots of both raster and vector data sources.
    * For most applications, best to avoid using a geographic projection (lat/lon) for analysis (or visualization) due to scaling issues
    * In general, for local studies or smaller areas (<500x500 km), you can use the appropriate UTM zone, which has acceptable distortion in terms of distance, angles and areas
    * For larger areas, probably want to define a custom projection or use common regional projection designed for your intended purpose
    * Be careful with the common Web Mercator projection - in the coming weeks we'll be using public tile datasets prepared with this projection, so try to remember some of the issues highlighted here

## Summary

OK, we covered a lot of ground with this one. These concepts are fundemental to all aspects of geospatial data analysis, and you will see them again. So if some aspects are still a bit fuzzy, know that there will be more opportunities to revisit and discuss.

* We covered basic operations, plotting, Geopandas GeoDataFrames and associated methods/attributes - these are great for most geospatial vector data and analysis. 
    * Note that standard GeoPandas operations may not be well-suited for workflows involving billions of points - these cases will require optimization (e.g., https://github.com/geopandas/dask-geopandas) and/or databases.
* Hopefully you got a sense of the most common approaches to define projections (EPSG codes, proj strings), and now understand some of the tradeoffs between different projections for different objectives involving visualization or quantitative analysis.

### Geometry objects
* POINT, LINE, POLYGON
* Polygon vs. MultiPolygon
* https://automating-gis-processes.github.io/site/notebooks/L1/geometric-objects.html

![Geometry types](https://datacarpentry.org/organization-geospatial/fig/dc-spatial-vector/pnt_line_poly.png)

### Geometry operations
* GEOS https://trac.osgeo.org/geos/, exposed by Shapely
    * 2D cartesian geometry operations - no knowledge of CRS
* https://geopandas.org/geometric_manipulations.html
    * Intersection
    * Union
    * Buffer

### Visualization: Chloropleth and Heatmap
* https://geopandas.org/docs/user_guide/mapping.html

### Plotting
* matplotlib
* holoviews/geoviews
* ipyleaflet, folium
* Basemap tiles with contextily

# Pandas

Trust me, you should learn how to use `Pandas`, regardless of your application.  

A significant portion of the Python data science ecosystem is based on Pandas and/or Pandas data models.

>pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with "relational" or "labeled" data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python. Additionally, it has the broader goal of becoming the most powerful and flexible open source data analysis / manipulation tool available in any language. It is already well on its way towards this goal.

https://github.com/pandas-dev/pandas#main-features

If you are working with tabular data (rows and columns, like a csv or spreadsheet), especially time series data, please use pandas.
* A better way to deal with tabular data, built on top of NumPy arrays
* With NumPy, we had to remember which column number (e.g., 3, 4) represented each variable in an array
* Pandas allows you to store data with different types, and then reference using more meaningful labels
    * NumPy: `glas_np[:,4]`
    * Pandas: `glas_df['glas_z']`
* A good "10-minute" reference with examples: https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

If you are working with more complex data, like collections of tabular time series data from 100s of met stations or netCDF model output, you can use [`xarray` package](http://xarray.pydata.org/en/stable/), which extends the `pandas` data model to n-dimensions.

# GeoPandas

pandas is great, but what if we want to do some geospatial operations - like reproject our vector data or compute the intersection between Point and Polygon features?

Enter Geopandas - all the great things about pandas, plus geo! (http://geopandas.org/).

>"GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and descartes and matplotlib for plotting."

>"GeoPandas enables you to easily do operations in python that would otherwise require a spatial database such as PostGIS."

Under the hood, GeoPandas is `pandas` plus some other core geospatial packages:
* `shapely` for geometry operations (https://shapely.readthedocs.io/en/stable/manual.html)
* `fiona` for reading/writing GIS file formats (https://fiona.readthedocs.io/en/latest/manual.html)
* `pyproj` for projections and coordinate system transformations (http://pyproj4.github.io/pyproj/stable/)

Under those hoods are lower-level geospatial libraries (GEOS, GDAL/OGR, PROJ4) that provide a foundation for most GIS software (open-source and commercial).  I encourage you to explore these - I guarantee you will learn something valuable.

* `GEOS` https://trac.osgeo.org/geos/
* `GDAL/OGR` https://gdal.org/
* `PROJ` https://proj.org/

For now, let's explore some basic geopandas functionality.

In [ ]:
import geopandas as gpd

In [ ]:
import hvplot.pandas
import holoviews as hv

### GeoDataFrame vs. GeoSeries
* https://geopandas.org/data_structures.html
* Indexing and selection - `iloc`, `loc`
* Pandas `squeeze`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.squeeze.html

In [ ]:
gpd.GeoDataFrame?

## Let's use some Polygons for US States

Hmmm, let's see.  Two choices:
1. We could go to ESRI or the U.S. Census website, identify and download a shapefile, unzip 4+ files, copy/paste the appropriate \*.shp filename into the notebook.  Wait, how can I download on a remote server?  OK, maybe run something like `wget http://...`, unzip, provide absolute path  
*- OR -*
2. Give geopandas a url string that points to a GeoJSON file somewhere on the web, and read dynamically

Yeah, let's go with #2

Let's use the US States 5M GeoJSON here: http://eric.clst.org/tech/usgeojson/

* JSON (JavaScript Object Notation) is a structured text-based data format. A Jupyter notebook is a json text file. 
* GeoJSON extends this format to include geospatial information. It's pretty great. If you are unfamiliar, take a moment to read about GeoJSON: https://en.wikipedia.org/wiki/GeoJSON

Take a look at the 5M file contents in your browser or download and open with a text editor.  Note organization structure.  How does this compare to, say, a Python dictionary object? 

This is a GeoJSON file!

Read the file using GeoPandas by passing the url to `gpd.read_file()`.

In [ ]:
states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_5m.json'
#states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json'
states_gdf = gpd.read_file(states_url)

In [ ]:
states_gdf.head()

In [ ]:
states_gdf.plot()

### Check the CRS
* Note that this was defined when we opened the file with GeoPandas - by default, a GeoJSON is assumed to use EPSG:4326 coordinate system for geodetic latitude and longitude

In [ ]:
states_gdf.crs

In [ ]:
states_gdf.hvplot()

In [ ]:
states_gdf.hvplot(geo=True)

# Part 4: Reprojection and Coordinate Systems

All of the above examples used standard geodetic lat/lon coordinate system (EPSG:4326).  This is fine for global analyses and basic visualization.  But remember that the width of a degree of longitude varies with latitude (~111 km near equator, ~0 m near pole), so our plots have scaling and aspect ratio issues.

We need to choose a map projection that is appropriate for our data. This decision is important for visualization, but is also critical for precise quantitative analysis. For example, if you want to compute area or volume change, you should use an equal-area projection. If you want to calculate distances between two points, you should use an equidistant projection.

https://www.axismaps.com/guide/general/map-projections/

Sadly, there is no "perfect" projection. You, as the mapmaker or data analyst, are responsible for choosing a projection with the right characteristics for your purposes. Let's explore a bit further, and we'll revisit some general guidelines later.

## Use GeoPandas to reproject your GeoDataFrame
* Use the very convenient `to_crs()` method to reproject: https://geopandas.org/projections.html
* Start by reprojecting the points to a Universal Transverse Mercator (UTM), Zone 11N (EPSG:32611)
* Store the output as a new GeoDataFrame
* Do a quick `head()` and note the new values in the `geometry` column

In [ ]:
states_gdf_utm = states_gdf.to_crs('EPSG:26912')

#### Note geometry difference

In [ ]:
states_gdf.head()

In [ ]:
states_gdf_utm.head()

In [ ]:
states_gdf.crs

In [ ]:
states_gdf_utm.crs

In [ ]:
states_gdf_utm.hvplot(aspect='equal') * hv.VLine(0).opts(color='k', line_width=0.5)

In [ ]:
states_gdf.to_crs('EPSG:26913').hvplot(aspect='equal') * hv.VLine(0).opts(color='k', line_width=0.5)

## Excellent, but what did we just do?

Under the hood, GeoPandas used the `pyproj` library (a Python API for PROJ) to transform each point from one coordinate system to another coordinate system.  

I guarantee that you've all done this kind of thing before, you may just not remember it or recognize it in this context. See: https://en.wikipedia.org/wiki/List_of_common_coordinate_transformations

In 2D, transforming (x,y) coordinates between different projections (e.g., WGS84 vs. UTM) on the same reference ellipsoid is pretty straightforward.  Things start to get more complicated when you include different ellipsoid models, horizontal/vertical datums, etc.  Oh, also the Earth's surface is not static - plate tectonics make everything more complicated, as time becomes important, and transformations must include a "kinematic" component.  

Fortunately, the `PROJ` library (https://proj.org/about.html) has generalized much of the complicated math for geodetic coordinate transformations.  It's been under development since the 1980s, and our understanding of the Earth's shape and plate motions has changed dramatically in that time period.  So, still pretty complicated, and there are different levels of support/sophistication in the tools/libraries that use `PROJ`.

We aren't going to get into the details here, but feel free to take a look at the Transformations section here to get a sense of how this is actually accomplished: https://proj.org/operations/index.html

The UTM projection we used above is not the best choice for our data extent, which span several UTM zones:
https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Utm-zones-USA.svg. 

We used Zone 12N, which means that our map will have limited distortion within that zone centered on -111°W.  But distortion will increase beyond the width of the -114° to -108°W zone definition.